# Statistics

In [1]:
import numpy as np
from scipy.stats import poisson as pois
from scipy.stats import norm
import matplotlib.pyplot as plt

import nbimporter
import prepData as prep
import fitFunc as fits

## Significance Test

We want to compute the significance of the observed dataset.  
The first steps are rescaling the dataset and fitting the background and signal functions.

We then compute the likelihood ratio of the observed data $q_0^{obs}$ between the signal and null hypothesis:

$$q_0^{obs} = -2 \cdot \log \left( \cfrac{\mathcal{L} \left(Data | 0, \hat{\theta}_0 \right)}{\mathcal{L} \left(Data | \hat{\mu}, \hat{\theta}_\hat{\mu} \right)}\right)$$

The ^ symbol indicates the values that optimize the fits.

In [2]:
# compute likelihood ratios of two hypotheses
def lh_ratio(y, null_vals, alt_vals, N=1365500):
    # compute log likelihoods
    LogLike_null = sum(norm.logpdf(x=y, loc=null_vals, scale=prep.calc_weights(null_vals, N)))
    LogLike_alt  = sum(norm.logpdf(x=y, loc=alt_vals,  scale=prep.calc_weights(alt_vals,  N)))
    
    #LogLike_null = sum(pois.logpmf(fft.astype(int), model_null.best_fit.astype(int)))
    #LogLike_alt  = sum(pois.logpmf(fft.astype(int), model_alt.best_fit.astype(int)))
    
    # ratio
    q = -2 * (LogLike_null - LogLike_alt)
    
    return q

The value of $q_0^{obs}$ has no meaning by itself, so we generate n = 10,000 toy datasets from the expected values given by the background fit and repeate the analysis for every new dataset. Both for the likelihood ratio and for the toy dataset generation, a normal approximation has been used instead of the formal Poisson distribution. 

In [3]:
def gen_toydataset(values, n, N=1365500):
    toy_dataset = norm.rvs(loc=values, scale=prep.calc_weights(values, N), size=(n, len(values)))
    
    #toy_dataset = pois.rvs(mu=values, size=(n,len(values)))
    
    return toy_dataset

The original $q_0^{obs}$ is thus compared with the distribution of $q_0$ obtained from the toy datasets, and the p-value is computed:

$$p_0 = P \left( q_0 \ge q_0^{obs} \right) = \int_{q_0^{obs}}^{+\infty} f(q_0 | 0, \hat{\theta}_0) \,dx $$ 

The significance is expressed as the number of $\sigma$s needed to achieve an equivalent p-value in a standard normal deviation:

$$z = \Phi^{-1} \left(1 - p_0 \right)$$  

In [4]:
def p_value(q_obs, q):
    p0 = sum(q >= q_obs)/len(q)
    return p0

This process is repeted using every possible frequency as $x_0$, the center of the signal function.

## Confidence Intervals

The process is similar to that of the significance test, but with a few key differences.  

In addition to fitting the background, we fit the signal twice: one time we let the $\mu$ run free to find $\hat{\mu}$, while the other we keep it fixed to a certain value $\tilde{\mu}$.  
The likelihood ratio is thus computed as:

$$q^{obs} \left( \tilde{\mu} \right) = -2 \cdot \log \left( \cfrac{\mathcal{L} \left(Data | \tilde{\mu}, \hat{\theta}_{\tilde{\mu}} \right)}{\mathcal{L} \left(Data | \hat{\mu}, \hat{\theta}_\hat{\mu} \right)}\right)$$

We then generate two sets of n = 10,000 toy datasets each, one as before from the expected values given by the background fit while the other from the signal fit with fixed $\tilde{\mu}$. We compare the original $q^{obs}(\tilde{\mu})$ with the distribution of $q(0)$ and $q(\tilde{\mu})$ from the toy datasets and compute the two probabilities:

$$    p_{\mu} = P \left(q(\tilde{\mu}) \ge q^{obs}(\tilde{\mu}) | \tilde{\mu} s + b \right)$$
$$1 - p_{b}   = P \left(q(0) \ge q^{obs}(\tilde{\mu}) | b \right)$$

and take their ratio.  
This process is done scanning different values of $\mu$ and we take as the 95% confidence interval limit the value of $\mu$ so that the ratio is equal to 0.05:

$$\mu^{95 \%CL} = \mu \; \big| \;CL_S = \cfrac{p_{\mu}}{1 - p_{b}} = 0.05$$

This process is repeted using every possible frequency as $x_0$, the center of the signal function.

## Code Implementation

In [5]:
def stat_test(run, x_0=np.array([]), mu_fix=np.array([]), signal=fits.signal_gauss, n_toy=10000,
              calc_z=False, calc_CI=False, draw=False, verbose=False, path='db/'):
    
    # load and prep data
    data, center, length = prep.load_dataset(run, path)
    freq, fft, weights, ref, N = prep.prep_data(data, center, length=length)
    
    # fit background once
    res_bkg = fits.fit_bkg(x=freq, y=fft, w=weights, center=center, ref=ref)
    bkg        = res_bkg.best_fit
    bkg_params = res_bkg.best_values
    
    # generate toy datasets from bkg and fit them
    toy_0 = gen_toydataset(values=bkg, n=n_toy, N=N)
        
        
    # scan x0 and perform desired tests
    z     = np.empty(len(x_0))
    mu_CI = np.empty(len(x_0))
    
    for i_x0 in range(len(x_0)):
        
        # fit signal
        sig = fits.fit_sig(x=freq, y=fft, w=weights, x_0=x_0[i_x0], init_params=bkg_params, signal=signal).best_fit
        
        # compute significance
        if calc_z:
            z[i_x0] = significance(freq, fft, bkg, sig, toy_0, center, ref, x_0[i_x0], signal, N, draw)
            
        # compute 95% CI mu
        if calc_CI:
            mu_CI[i_x0] = CI(freq, fft, weights,
                             bkg_params, center, ref,
                             x_0[i_x0], signal, mu_fix, sig,
                             toy_0, N, draw, verbose)
            
        if (i_x0+1)%20 == 0:
            print("Step:", i_x0+1)
            
    return z, mu_CI

In [6]:
def significance(x, y, bkg, sig, toy_0, center, ref, x_0, signal, N=1365500, draw=False):
    
    # compute likelihood ratio of observed data
    q0_obs = lh_ratio(y, bkg, sig, N)     
    
    n_toy = len(toy_0)
    
    # analyze toy datasets
    q0 = np.empty(n_toy)
    for i_toy in range(n_toy):
        # fit background over toy dataset
        toy_w = prep.calc_weights(toy_0[i_toy], N)
        res_toy_bkg = fits.fit_bkg(x=x, y=toy_0[i_toy], w=toy_w, center=center, ref=ref)
        toy_bkg = res_toy_bkg.best_fit
        toy_bkg_params = res_toy_bkg.best_values
        # fit signal
        toy_sig = fits.fit_sig(x=x, y=toy_0[i_toy], w=toy_w, x_0=x_0,
                               init_params=toy_bkg_params, signal=signal).best_fit
      
        # compue likelihood ratio of toy dataset 
        q0[i_toy] = lh_ratio(toy_0[i_toy], toy_bkg, toy_sig, N)
    
    # plot significance distribution
    if(draw):
        plot_lhratio(q0_obs, q0)
        
    # compute significance
    p0 = p_value(q0_obs, q0)
    z = norm.ppf(1-p0)
        
    return z

In [7]:
def CI(x, y, w, bkg_params, center, ref, x_0, signal, mu_fix, sig, toy_0, N=1365500, draw=False, verbose=False):
    
    n_toy = len(toy_0)
    
    # scan for mu
    r = np.zeros(len(mu_fix))
    for i_mu in range(len(mu_fix)): 
        
        fix = fits.fit_sig(x, y, w, x_0, bkg_params, signal, mu_init=mu_fix[i_mu], mu_vary=False).best_fit
        
        # compute likelihood ratio of observed data
        q_mu_obs = lh_ratio(y, fix, sig)
        
        # generate toy datasets from fixed mu
        toy_fix = gen_toydataset(fix, n_toy, N)
        
        # comute distriution of likelihood ratios
        q_mu = calc_qmu(x, toy_fix, center, ref, x_0, mu_fix[i_mu], signal, N)
        q0   = calc_qmu(x, toy_0,   center, ref, x_0, mu_fix[i_mu], signal, N)
        
        # compute p-values
        p_mu = p_value(q_mu_obs, q_mu)
        p_b  = p_value(q_mu_obs, q0)
        
        # compute ratio
        r[i_mu] = p_mu/p_b
        
        if(verbose):
            print("Mu: ", mu_fix[i_mu], "\np_mu =", p_mu, "  p_b =", p_b, "  ratio =", r[i_mu], "\n")
            
        # plot significance distribution
        if(draw):
            plot_lhratio(q_mu_obs, q0, q_mu)
    
    idx = np.argmin(np.abs(r - 0.05))
        
    return(mu_fix[idx])

In [8]:
def calc_qmu(x, toy, center, ref, x_0, mu_fix, signal, N=1365500):
    
    n_toy = len(toy)
    
    # compute likelihood ratio for toy dataset
    q_mu = np.empty(n_toy)
    for i_toy in range(n_toy):
        # compute signal and fixed mu signal over toy dataset
        toy_w = prep.calc_weights(toy[i_toy], N)
        toy_bkg_params = fits.fit_bkg(x, toy[i_toy], toy_w, center, ref).best_values
        toy_fix = fits.fit_sig(x, toy[i_toy], toy_w, x_0, toy_bkg_params, signal,
                               mu_init=mu_fix, mu_vary=False).best_fit
        toy_sig = fits.fit_sig(x, toy[i_toy], toy_w, x_0, toy_bkg_params, signal).best_fit
            
        # likelihood ratio
        q_mu[i_toy] = lh_ratio(toy[i_toy], toy_fix, toy_sig, N)
        
    return q_mu

In [9]:
def plot_lhratio(q_obs, q0, q_alt = np.array([])):
    # prepare canvas
    fig = plt.figure(figsize=(15,10))
    
    # plot q0 distribution
    #N = len(q0)
    #binning = int(np.sqrt(N))
    n, bins, _ = plt.hist(q0, bins="auto", density=True, alpha=0.5,
                          facecolor='lightblue', edgecolor='black', label='Toy Experiments: background')
    lineheight = max(n)
    if np.any(q_alt):
        n_alt, bins_alt, _ = plt.hist(q_alt, bins="auto", density=True, alpha=0.5,
                                      facecolor='salmon', edgecolor='black', label='Toy Experiments: signal')
        lineheight = max(lineheight, max(n_alt))
    
    plt.vlines(q_obs, 0, lineheight, colors='forestgreen', linestyles='--', label='Observed Data')
    
    plt.legend(loc='upper right')
    plt.xlabel('q')
    plt.ylabel('PDF')
    
    plt.show()